# 2.1.1 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()

In [3]:
# Zeichenerklärung (BOUNDING BOX, CROPPED)
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-bb-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg4 = jarvis.request([
    {
        "type": "text",
        "text": 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg4)

image_size: [728, 2048]
Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+-------------------+-----------------------+
| Baugebiet         | Zahl der Vollgeschosse|
+-------------------+-----------------------+
| Grundflächenzahl  | Geschossflächenzahl   |
+-------------------+-----------------------+
| -                 | Bauweise              |
+-------------------+-----------------------+
| Dachform und      | Dachneigung           |
+-------------------+-----------------------+
```


In [4]:
# Zeichenerklärung (BOUNDING BOX, CROPPED)
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-bb-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

jarvis.withContext([
    {
        "type": "text",
        "text": msg4
    }
])

msg5 = jarvis.request([
    {
        "type": "text",
        "text": 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Parameter>: {"Definition": "", "Beispiel"=""}}'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg5)

image_size: [728, 2048]
```json
{
  "Baugebiet": {
    "Definition": "Art des Baugebiets gemäß den Festsetzungen des Bebauungsplans.",
    "Beispiel": "Reines Wohngebiet (WR), Allgemeines Wohngebiet (WA), Mischgebiet (MI)"
  },
  "Zahl der Vollgeschosse": {
    "Definition": "Anzahl der oberirdischen Geschosse, die ein Gebäude haben darf.",
    "Beispiel": "2, 3, 4"
  },
  "Grundflächenzahl (GRZ)": {
    "Definition": "Verhältnis der bebauten Fläche zur Grundstücksfläche.",
    "Beispiel": "0,4, 0,3"
  },
  "Geschossflächenzahl (GFZ)": {
    "Definition": "Verhältnis der gesamten Geschossfläche zur Grundstücksfläche.",
    "Beispiel": "1,2, 0,8"
  },
  "Bauweise": {
    "Definition": "Art und Weise der Bebauung, z.B. offene oder geschlossene Bauweise.",
    "Beispiel": "offene Bauweise, geschlossene Bauweise"
  },
  "Dachform und Dachneigung": {
    "Definition": "Vorgaben zur Form und Neigung des Daches.",
    "Beispiel": "Satteldach, Flachdach, 30° Neigung"
  }
}
```
